### Imports

In [ ]:
import os
import sys
sys.path.append('../scripts')

import modis
import utils

import rasterio
import rioxarray as rxr
import earthpy.plot as ep


### Setup

In [ ]:
raw_dir = "../data/modis"
filename = "MOD14A1.A2021209.h10v03.061.2021218013045.hdf"
processing_dir = "../data/modis/processing"
file_path = os.path.join(raw_dir, filename)

### Extracting the fire mask

In [ ]:
firemask, dates, fire_filename = modis.extract_fire_mask(file_path)


In [ ]:
fire = rasterio.open(fire_filename)
ep.plot_bands(fire.read(1), title="Fire", cmap="YlOrRd", figsize=(10, 10))

### Cropping based on a bounding box

In [ ]:
coords, pixels = modis.get_coords_and_pixels(fire)
bbox_coords = utils.create_bounding_box(coords)
modis_cropped_path = modis.crop(fire_filename, bbox_coords)

In [ ]:
# open the cropped MODIS image
cropped_modis = rasterio.open(modis_cropped_path)

# Plot the clipped MODIS image
ep.plot_bands(cropped_modis.read(1) > 6, title="Clipped MODIS Image", cmap="YlOrRd")

### Resizing the modis file

In [ ]:
file = modis.resize(modis_cropped_path)

In [ ]:
# Open the resized MODIS image
tiff_file = rxr.open_rasterio(file)
tiff_file.shape